# wps_CI_on_demand_with_indices

#### wps_CI is a process that runs the [ci.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CI.R#L235) (Climate Imprint) function of the [ClimDown](https://cran.r-project.org/web/packages/ClimDown/index.html) package. This notebook uses either the PNWNAmet daily gridded observations or the CMIP6 downscaled data as the `gcm_file` parameter and the PRISM monthly climatologies as the `obs_file` parameter. By supplying climatologies, this skips the computation of the climatologies during Climate Imprint. After running CI, the output is passed to the Ouranos `finch` bird to compute climate indices. To get started, first instantiate the clients to the two birds. Here, the clients will try to connect to a `chickadee`/`finch` instances using the `url` parameter. This instantiation also takes advantage of asynchronous execution by setting `progress` to True.

## 1. Setup chickadee/finch instance and input parameters

I. Import Python packages used throughout the notebook.

In [60]:
import os
import shapely.geometry
import numpy as np
import geopandas as gpd
import xarray as xr
from birdy import WPSClient
from tempfile import NamedTemporaryFile
from netCDF4 import Dataset
from datetime import datetime
from requests_html import HTMLSession
from ipywidgets import *
from ipyleaflet import *

II. Connect to the chickadee/finch instances given by the `urls`.

In [3]:
# NBVAL_IGNORE_OUTPUT
chickadee_url = "http://docker-dev03.pcic.uvic.ca:30102"
print(f"Using chickadee on {chickadee_url}")
chickadee = WPSClient(chickadee_url, progress=True)
finch_url = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/finch/wps"
print(f"Using finch on {finch_url}")
finch = WPSClient(finch_url, progress=True)

Using chickadee on http://docker-dev03.pcic.uvic.ca:30102
Using finch on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/finch/wps


III. Display help for individual processes by using the ? command (ex. bird.process?). We can use the docstring to ensure we provide the appropriate parameters.

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.ci?

Signature:
chickadee.ci(
    gcm_file,
    obs_file,
    gcm_varname=None,
    obs_varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    out_file=None,
)
Docstring:
Climate Imprint (CI) downscaling

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
units_boo

In [5]:
# NBVAL_IGNORE_OUTPUT
finch.tx_max?

Signature:
finch.tx_max(
    tasmax=None,
    freq='YS',
    month=None,
    season=None,
    check_missing='any',
    missing_options=None,
    cf_compliance='warn',
    data_validation='raise',
    variable=None,
    output_name=None,
    output_format='netcdf',
    csv_precision=None,
    output_formats=None,
)
Docstring:
The maximum value of daily maximum temperature.

Parameters
----------
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF Files or archive (tar/zip) containing netCDF files. Maximum surface temperature.
freq : {'YS', 'MS', 'QS-DEC', 'AS-JUL'}string
    Resampling frequency.
month : {'1', '2', '3', '4', '5', '6', '7', '8', '9', '10', ...}integer
    Months of the year over which to compute indicator.
season : {'DJF', 'MAM', 'JJA', 'SON'}string
    Climatological season over which to compute indicator.
check_missing : {'any', 'wmo', 'pct', 'at_least_n', 'skip', 'from_context'}string
    Method used to determine which a

IV. Set up an interactive map to initialize the inputs. These inputs are the <b>center point</b> of the 2.5 degree by 2.5 degree subdomain of the target region and 3 degree by 3 degree subdomain of the GCM region, the <b>climate variable</b> from the GCM input to downscale, the <b>dataset</b> to use as the GCM input (where choosing the CMIP6 data allows one to further select the <b>downscaling technique</b>, the <b>model</b>, the <b>CanESM5 run</b> if applicable, and the <b>emissions scenario</b>), and the <b>climatological period</b> of the target data. Some inputs are obtained from the [THREDDS data server](https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog.html).

In [6]:
sub_layers = LayerGroup()
thredds_base = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets"
thredds_catalog = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog/datasets"

def in_bc(point):
    bc = f"{thredds_base}/storage/data/climate/PRISM/dataportal/pr_monClim_PRISM_historical_run1_197101-200012.nc"
    bc_data = Dataset(bc)
    bc_lat = bc_data.variables["lat"][:]
    bc_lon = bc_data.variables["lon"][:]
    # Check if center point is within lat/lon grid
    if (point[0] < bc_lat[0]) or (point[0] > bc_lat[-1]) or (point[1] < bc_lon[0]) or (point[1] > bc_lon[-1]):
        return False
    # Check if point is closest to a masked data value
    else:
        lat_index = np.argmin(np.abs(bc_lat - point[0]))
        lon_index = np.argmin(np.abs(bc_lon - point[1]))
        pr = bc_data.variables["pr"][0, lat_index, lon_index]
        if pr.mask:
            return False
    return True
    
def get_subdomain(lat_min, lat_max, lon_min, lon_max, color, name):
    coords = [(lat_min, lon_min), (lat_max, lon_max)]
    return Rectangle(bounds=coords, color=color, name=name, draggable=True)

def get_models():
    session = HTMLSession()
    r = session.get(f"{thredds_catalog}/storage/data/climate/downscale/BCCAQ2/CMIP6_BCCAQv2/catalog.html")
    models = []
    exclude = ["CMIP6_BCCAQv2", "CWEC2020_Factors/", "Degree_Climatologies/", "Ensemble_Averages/", "nobackup/", "--", ""]
    for tt in r.html.find("tt"):
        if tt.text not in exclude:
            models.append(tt.text[:-1])
    models.sort()
    return models

def handle_dataset_change(change):
    technique.disabled = not technique.disabled
    model.disabled = not model.disabled
    scenario.disabled = not scenario.disabled

def handle_model_change(change):
    if model.value == "CanESM5":
        canesm5_run.disabled = False
    else:
        canesm5_run.disabled = True
        
def handle_interact(**kwargs):
    point = (round(kwargs.get("coordinates")[0], 5), round(kwargs.get("coordinates")[1], 5))
    center_hover.value = str(point)
    if kwargs.get("type") == "click":
        # Check if point is within PRISM region
        if not in_bc(point):
            return
        # Remove previous center point and subdomains
        for layer in sub_layers.layers:
            sub_layers.remove_layer(layer)

        # Add new subdomains
        m.center_point = point
        center.value = str(m.center_point)
        center_marker = Marker(location=m.center_point, name="Marker")
        
        m.lat_min_obs, m.lat_max_obs = (m.center_point[0] - 1.25, m.center_point[0] + 1.25)
        m.lon_min_obs, m.lon_max_obs = (m.center_point[1] - 1.25, m.center_point[1] + 1.25)
        m.lat_min_gcm, m.lat_max_gcm = (m.center_point[0] - 1.5, m.center_point[0] + 1.5)
        m.lon_min_gcm, m.lon_max_gcm = (m.center_point[1] - 1.5, m.center_point[1] + 1.5)

        gcm_subdomain = get_subdomain(m.lat_min_gcm, m.lat_max_gcm, m.lon_min_gcm, m.lon_max_gcm, "blue", "GCM")
        obs_subdomain = get_subdomain(m.lat_min_obs, m.lat_max_obs, m.lon_min_obs, m.lon_max_obs, "red", "Obs")

        sub_layers.add_layer(center_marker)
        sub_layers.add_layer(gcm_subdomain)
        sub_layers.add_layer(obs_subdomain)
        m.add_layer(sub_layers)

In [7]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = "Default"

m = Map(
    basemap=mapnik,
    center=(53.5, -120),
    zoom=5,
    layout=Layout(height="600px"),
)
m.on_interaction(handle_interact)
m.center_point = ()

#df = gpd.read_file('/home/eyvorchuk/code/bc_layers/BEC_.shp')
#geodata = GeoData(geo_dataframe=df)
#m.add_layer(geodata)

legend = LegendControl({"GCM": "blue", "Obs": "red"}, name="Subdomains", position="topright")
m.add_control(legend)

center_hover = Text(value="", placeholder="") 
center = Text(value="", placeholder="", description="Center:")
vars = RadioButtons(options=["pr", "tasmax", "tasmin"], description="Climate variable:")
dataset = RadioButtons(options=["PNWNAmet", "CMIP6"], description="Dataset:")
dataset.observe(handle_dataset_change)

technique = RadioButtons(options=["BCCAQv2", "MBCn"], description="CMIP6 downscaling technique:", disabled = True)
model = Dropdown(options=get_models(), description="CMIP6 model:", disabled = True)
model.style.description_width = "100px"
model.observe(handle_model_change)

canesm5_runs = ["r" + str(r) + "i1p2f1" for r in range(1, 11)]
canesm5_run = Dropdown(options=canesm5_runs, description="CanESM5 run:", disabled = True)
canesm5_run.style.description_width = "100px"

scenario = RadioButtons(options=[("SSP1-2.6", "ssp126"), ("SSP2-4.5", "ssp245"), ("SSP5-8.5", "ssp585")], description="CMIP6 emissions scenario:", disabled = True)
period = RadioButtons(options=["197101-200012", "198101-201012"], description="Climatological period:")

box_layout = Layout(display='flex',
                flex_flow = 'column', 
                width='110%',
                align_items = 'center')
control_box = Box(children = [center_hover, center, vars, dataset, technique, model, canesm5_run, scenario, period], layout=box_layout)
AppLayout(center = m, right_sidebar = control_box, align_items = 'center')

AppLayout(children=(Box(children=(Text(value='', placeholder=''), Text(value='', description='Center:', placeh…

V. Obtain the input data files from the [THREDDS data server](https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog.html) and examine their structures.

In [8]:
data_vars = {"pr": "pr", "tasmax": "tmax", "tasmin": "tmin"}

gcm_var = vars.value
obs_var = data_vars[gcm_var]

if dataset.value == "PNWNAmet":
    gcm_file = f"{thredds_base}/storage/data/projects/dataportal/data/vic-gen2-forcing/PNWNAmet_{gcm_var}_invert_lat.nc"
else:
    if technique.value == "BCCAQv2":
        technique_dir = "BCCAQ2"
        model_dir = model.value
    else:
        technique_dir = "MBCn"
        model_dir = model.value + "_10"
    model_catalog = f"{thredds_catalog}/storage/data/climate/downscale/{technique_dir}/CMIP6_{technique.value}/{model_dir}/catalog.html"
    
    session = HTMLSession()
    r = session.get(model_catalog)
    for tt in r.html.find("tt"):
        file = tt.text
        if (gcm_var in file) and (scenario.value in file):
            if (model.value == "CanESM5") and (canesm5_run.value not in file):
                continue
            break
    gcm_file = f"{thredds_base}/storage/data/climate/downscale/{technique_dir}/CMIP6_{technique.value}/{model_dir}/{file}"
        
obs_file = f"{thredds_base}/storage/data/climate/PRISM/dataportal/{obs_var}_monClim_PRISM_historical_run1_{period.value}.nc"
print("GCM File: " + gcm_file + "\n")
print("Obs File: " + obs_file + "\n")
gcm_dataset = Dataset(gcm_file)
obs_dataset = Dataset(obs_file)
print("GCM Structure: " + str(gcm_dataset.dimensions.items()) + "\n")
print("Obs Structure: " + str(obs_dataset.dimensions.items()) + "\n")

GCM File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/dataportal/data/vic-gen2-forcing/PNWNAmet_tasmax_invert_lat.nc

Obs File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/climate/PRISM/dataportal/tmax_monClim_PRISM_historical_run1_197101-200012.nc

GCM Structure: dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 24837), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 512), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1088)])

Obs Structure: dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 13), ('bnds', <class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 1680), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 3241), ('maxStrlen64', <class 'netCDF4._netCDF4.Dimension'>

VI. Store the datasets' latitudes and longitudes into variables and examine their values.

In [9]:
gcm_lats = gcm_dataset.variables["lat"][:]
gcm_lons = gcm_dataset.variables["lon"][:]
obs_lats = obs_dataset.variables["lat"][:]
obs_lons = obs_dataset.variables["lon"][:]

In [10]:
gcm_lats

masked_array(data=[40.03125, 40.09375, 40.15625, 40.21875, 40.28125,
                   40.34375, 40.40625, 40.46875, 40.53125, 40.59375,
                   40.65625, 40.71875, 40.78125, 40.84375, 40.90625,
                   40.96875, 41.03125, 41.09375, 41.15625, 41.21875,
                   41.28125, 41.34375, 41.40625, 41.46875, 41.53125,
                   41.59375, 41.65625, 41.71875, 41.78125, 41.84375,
                   41.90625, 41.96875, 42.03125, 42.09375, 42.15625,
                   42.21875, 42.28125, 42.34375, 42.40625, 42.46875,
                   42.53125, 42.59375, 42.65625, 42.71875, 42.78125,
                   42.84375, 42.90625, 42.96875, 43.03125, 43.09375,
                   43.15625, 43.21875, 43.28125, 43.34375, 43.40625,
                   43.46875, 43.53125, 43.59375, 43.65625, 43.71875,
                   43.78125, 43.84375, 43.90625, 43.96875, 44.03125,
                   44.09375, 44.15625, 44.21875, 44.28125, 44.34375,
                   44.40625, 44.46

In [11]:
gcm_lons

masked_array(data=[-168.96875, -168.90625, -168.84375, ..., -101.15625,
                   -101.09375, -101.03125],
             mask=False,
       fill_value=1e+20)

In [12]:
obs_lats

masked_array(data=[48.        , 48.00833333, 48.01666667, ...,
                   61.975     , 61.98333333, 61.99166667],
             mask=False,
       fill_value=1e+20)

In [13]:
obs_lons

masked_array(data=[-140.        , -139.99166667, -139.98333333, ...,
                   -113.01666667, -113.00833333, -113.        ],
             mask=False,
       fill_value=1e+20)

VII. Request a subset of each dataset based on each subdomain. This is done by determining the indices corresponding to the min/max lat and lon values and using those in the THREDDS requests. The full time range for each dataset is also used.

In [14]:
def get_index_range(arr, min_val, max_val):
    """Compute the indices in an array that correspond to the array's values
    closest to desired min/max values."""        
    min_index = np.argmin(np.abs(arr - min_val))
    max_index = np.argmin(np.abs(arr - max_val))
    return (min_index, max_index)

In [15]:
gcm_lat_indices = get_index_range(gcm_lats, m.lat_min_gcm, m.lat_max_gcm)
gcm_lon_indices = get_index_range(gcm_lons, m.lon_min_gcm, m.lon_max_gcm)
obs_lat_indices = get_index_range(obs_lats, m.lat_min_obs, m.lat_max_obs)
obs_lon_indices = get_index_range(obs_lons, m.lon_min_obs, m.lon_max_obs)

In [16]:
print("GCM lat index range: " + str(gcm_lat_indices))
print("GCM lon index range: " + str(gcm_lon_indices))
print("Obs lat index range: " + str(obs_lat_indices))
print("Obs lon index range: " + str(obs_lon_indices))

GCM lat index range: (218, 266)
GCM lon index range: (679, 727)
Obs lat index range: (711, 1011)
Obs lon index range: (1643, 1943)


In [17]:
gcm_lat_range = f"[{gcm_lat_indices[0]}:{gcm_lat_indices[1]}]"
gcm_lon_range = f"[{gcm_lon_indices[0]}:{gcm_lon_indices[1]}]"
obs_lat_range = f"[{obs_lat_indices[0]}:{obs_lat_indices[1]}]"
obs_lon_range = f"[{obs_lon_indices[0]}:{obs_lon_indices[1]}]"

In [18]:
gcm_ntime = len(gcm_dataset.variables["time"][:])
gcm_time_range = f"[0:{gcm_ntime - 1}]"
obs_ntime = len(obs_dataset.variables["time"][:])
obs_time_range = f"[0:{obs_ntime - 1}]"

In [19]:
gcm_subset_file = f"{gcm_file}?time,lat{gcm_lat_range},lon{gcm_lon_range},{gcm_var}{gcm_time_range}{gcm_lat_range}{gcm_lon_range}"
obs_subset_file = f"{obs_file}?time,lat{obs_lat_range},lon{obs_lon_range},climatology_bounds,crs,{obs_var}{obs_time_range}{obs_lat_range}{obs_lon_range}"
print("GCM Subset: " + gcm_subset_file + "\n")
print("Obs Subset: " + obs_subset_file + "\n")
gcm_subset_dataset = Dataset(gcm_subset_file)
obs_subset_dataset = Dataset(obs_subset_file)
print("GCM Subset Structure: " + str(gcm_subset_dataset.dimensions.items()) + "\n")
print("Obs Subset Structure: " + str(obs_subset_dataset.dimensions.items()))

GCM Subset: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/dataportal/data/vic-gen2-forcing/PNWNAmet_tasmax_invert_lat.nc?time,lat[218:266],lon[679:727],tasmax[0:24836][218:266][679:727]

Obs Subset: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/climate/PRISM/dataportal/tmax_monClim_PRISM_historical_run1_197101-200012.nc?time,lat[711:1011],lon[1643:1943],climatology_bounds,crs,tmax[0:12][711:1011][1643:1943]

GCM Subset Structure: dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 24837), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 49), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 49)])

Obs Subset Structure: dict_items([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 13), ('bnds', <class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2), ('lat', <class 'netCDF4._netCDF

In [20]:
print("GCM subset lats: " + str(gcm_subset_dataset.variables["lat"][:]) + "\n")
print("GCM subset lons: " + str(gcm_subset_dataset.variables["lon"][:]))

GCM subset lats: [53.65625 53.71875 53.78125 53.84375 53.90625 53.96875 54.03125 54.09375
 54.15625 54.21875 54.28125 54.34375 54.40625 54.46875 54.53125 54.59375
 54.65625 54.71875 54.78125 54.84375 54.90625 54.96875 55.03125 55.09375
 55.15625 55.21875 55.28125 55.34375 55.40625 55.46875 55.53125 55.59375
 55.65625 55.71875 55.78125 55.84375 55.90625 55.96875 56.03125 56.09375
 56.15625 56.21875 56.28125 56.34375 56.40625 56.46875 56.53125 56.59375
 56.65625]

GCM subset lons: [-126.53125 -126.46875 -126.40625 -126.34375 -126.28125 -126.21875
 -126.15625 -126.09375 -126.03125 -125.96875 -125.90625 -125.84375
 -125.78125 -125.71875 -125.65625 -125.59375 -125.53125 -125.46875
 -125.40625 -125.34375 -125.28125 -125.21875 -125.15625 -125.09375
 -125.03125 -124.96875 -124.90625 -124.84375 -124.78125 -124.71875
 -124.65625 -124.59375 -124.53125 -124.46875 -124.40625 -124.34375
 -124.28125 -124.21875 -124.15625 -124.09375 -124.03125 -123.96875
 -123.90625 -123.84375 -123.78125 -123.71875 -1

In [20]:
print("Obs subset lats: " + str(obs_subset_dataset.variables["lat"][:]) + "\n")
print("Obs subset lons: " + str(obs_subset_dataset.variables["lon"][:]))

Obs subset lats: [55.38333333 55.39166667 55.4        55.40833333 55.41666667 55.425
 55.43333333 55.44166667 55.45       55.45833333 55.46666667 55.475
 55.48333333 55.49166667 55.5        55.50833333 55.51666667 55.525
 55.53333333 55.54166667 55.55       55.55833333 55.56666667 55.575
 55.58333333 55.59166667 55.6        55.60833333 55.61666667 55.625
 55.63333333 55.64166667 55.65       55.65833333 55.66666667 55.675
 55.68333333 55.69166667 55.7        55.70833333 55.71666667 55.725
 55.73333333 55.74166667 55.75       55.75833333 55.76666667 55.775
 55.78333333 55.79166667 55.8        55.80833333 55.81666667 55.825
 55.83333333 55.84166667 55.85       55.85833333 55.86666667 55.875
 55.88333333 55.89166667 55.9        55.90833333 55.91666667 55.925
 55.93333333 55.94166667 55.95       55.95833333 55.96666667 55.975
 55.98333333 55.99166667 56.         56.00833333 56.01666667 56.025
 56.03333333 56.04166667 56.05       56.05833333 56.06666667 56.075
 56.08333333 56.09166667 56.1  

VIII. Put together the parameters for `chickadee.ci`. In the case for `pr`, the `units_bool` parameter is set to `False` in order to avoid converting the PRISM's `mm` units to the PNWNAmet's `mm/day` units.

In [44]:
(start_date, end_date) = period.value.split("-")
start_date = datetime.strptime(start_date, "%Y%m")
end_date = datetime.strptime(end_date + "31", "%Y%m%d")
chickadee_params = {"gcm_file": gcm_subset_file, "obs_file": obs_subset_file, "gcm_varname": gcm_var, "obs_varname": obs_var, "max_gb": 0.5, 
          "start_date": start_date, "end_date": end_date}
if gcm_var == "pr":
    chickadee_params["units_bool"] = False
    chickadee_params["pr_units"] = "mm/day"

## 2. Run `chickadee.ci` and use the output to run `finch.tx_max`.

I. Run `chickadee.ci`.

In [46]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True) as out_file:
    chickadee_params["out_file"] = out_file.name
    chickadee_output = chickadee.ci(**chickadee_params)

ERROR:root:Could not read status document.
ERROR:root:Could not parse XML response.


II. (Optionally) download the output directly from the WPS output URL.

In [50]:
chickadee_output_url = chickadee_output.get()[0]
print("Download output from this URL: " + chickadee_output_url)

Download output from this URL: https://docker-dev03.pcic.uvic.ca/wpsoutputs/23a54182-a02e-11ee-b6d1-0242ac12000a/output_xug1uobr.nc


III. Run `finch.tx_max`.

In [48]:
def get_finch_output(resp):
    """Get the URL of the Finch output file for downloading."""
    print("Process status: ", resp.status)
    urls = resp.get()
    print("Link to process output: ", urls.output)

In [58]:
def finch_output_to_temp(resp, var):
    """Store Finch output in temporary file and open using xarray. This returns the xarray object so that it can be further examined."""
    with NamedTemporaryFile() as fn:
        resp.getOutput(fn.name, identifier="output")
        ds = xr.open_dataset(fn.name, decode_timedelta=False)
        print(str(getattr(ds, var)))
        return ds

In [53]:
chickadee_output_thredds = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/birdhouse_wps_outputs/"
chickadee_output_thredds += chickadee_output_url.split("wpsoutputs/")[1]
print("THREDDS URL for chickadee output: " + chickadee_output_thredds)

THREDDS URL for chickadee output: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/birdhouse_wps_outputs/23a54182-a02e-11ee-b6d1-0242ac12000a/output_xug1uobr.nc


In [54]:
tx_max_ys = finch.tx_max(chickadee_output_thredds, output_name="tx_max_ys")

ERROR:root:Could not read status document.
ERROR:root:Could not parse XML response.


In [56]:
get_finch_output(tx_max_ys)

Process status:  ProcessSucceeded
Link to process output:  https://docker-dev03.pcic.uvic.ca/wpsoutputs/finch/cc0998c0-a031-11ee-ad39-0242ac120008/tx_max_ys.nc


In [61]:
tx_max_ys_arr = finch_output_to_temp(tx_max_ys, "tx_max")

<xarray.DataArray 'tx_max' (time: 68, lat: 301, lon: 301)>
[6160868 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1945-01-01 1946-01-01 ... 2012-01-01
  * lon      (lon) float64 -126.3 -126.3 -126.3 -126.3 ... -123.8 -123.8 -123.8
  * lat      (lat) float64 53.92 53.93 53.94 53.95 ... 56.4 56.41 56.42 56.42
Attributes:
    units:          K
    cell_methods:    time: maximum over days
    history:        [2023-12-21 18:50:32] tx_max: TX_MAX(tasmax=tasmax, freq=...
    standard_name:  air_temperature
    long_name:      Maximum daily maximum temperature
    description:    Annual maximum of daily maximum temperature.
